# Go Through Each Patient and Create a Cellbender Output Folder

In [ ]:
import pandas as pd
import os
import subprocess

In [ ]:
df.head()

,Unnamed: 0,SynID,FileName,LibraryID
0,0,syn32164304,190403-B4-A_Broad_S1_L007_I1_001.fastq.gz,190403-B4-A
1,1,syn32164319,190403-B4-A_Broad_S1_L007_R1_001.fastq.gz,190403-B4-A
2,2,syn32164389,190403-B4-A_Broad_S1_L007_R2_001.fastq.gz,190403-B4-A
3,3,syn32164308,190403-B4-B_Broad_S1_L008_I1_001.fastq.gz,190403-B4-B
4,4,syn32164325,190403-B4-B_Broad_S1_L008_R1_001.fastq.gz,190403-B4-B


In [ ]:
root = '/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/DeJager/Preprocessed_Counts'
df = pd.read_csv('/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/DeJager/FASTQs_Download/FASTQ_Download_CSVs/Synapse_FASTQ_IDs.csv')

LibraryID = set(df['LibraryID'])

#Commented the code below out as it should only be run once to create the output folders

for num in LibraryID:
    os.mkdir(os.path.join(root, str(num)))

# Generate Batch Scripts For Cellbender

In [ ]:
parent_scripts = '/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/DeJager/Cellbender/Batch_Scripts'
root_dir = '/om/scratch/Mon/mabdel03/Counts'

for num in LibraryID:

    inp_dir = os.path.join(root_dir, str(num)+'/outs')
    inp_file = os.path.join(inp_dir, 'raw_feature_bc_matrix.h5')
    out_dir = os.path.join(root, str(num))
    out_file = os.path.join(out_dir, 'processed_feature_bc_matrix.h5')

    output = f"""#!/bin/bash
#SBATCH -n 32                    # Number of cores requested
#SBATCH -t 47:00:00             # Runtime in minutes or
#SBATCH --output=/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/DeJager/Cellbender/Batch_Scripts_Outs/slurm-%j.out
#SBATCH --error=/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/DeJager/Cellbender/Batch_Scripts_Err/slurm-%j.err
#SBATCH --gres=gpu:a100:1
#SBATCH --mem=128G
#SBATCH --mail-user=mabdel03@mit.edu
#SBATCH --mail-type=FAIL

source /orcd/data/lhtsai/001/om2/mabdel03/miniforge3/etc/profile.d/conda.sh

conda activate /orcd/data/lhtsai/001/om2/mabdel03/conda_envs/Cellbender_env

cd {out_dir}

cellbender remove-background --cuda --input {inp_file} --fpr 0 --output {out_file}
"""
    filename = str(num)+'_cellbender.sh'
    f = open(os.path.join(parent_scripts, filename), 'x') #make the file
    f.write(output) #write to the file
    f.close()
    sbatch_command = f'sbatch {os.path.join(parent_scripts, filename)}' #submit sbatch command
    process = subprocess.Popen(sbatch_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

